## Теоретическая часть

1. Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. Чем они отличаются и чем они похожи? Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?  
В чем принципиальные отличия item-item рекомендаций от ALS?


С точки зрения математики, item_item и user_user идентичны. Различаются "направлением" данных, что у нас представляет столбец в датафрейме - товар, либо пользователя. Другими словами, если единственным действием для преобразования одного алгоритма в другой, будет транспонирование исходной матрицы. 

Что до различия с точки зрения бизнеса, user_user будет хорошо отрабатывать для "постоянного" клиента, о котором уже собраны данные. Плохо - для нового. На покойном ныне Имхонете, судя по всему, применялся этот подход. item_item - наоборот, мало учитывает персонификацию, и предлагает товары на основе "похожести", "совместных покупок", новые товары по этой логике будут непопулярны.

ALS, в отличие от предыдущих двух алгоритмов, разбивает большую разреженную матрицу на две "узких", экономя место для хранения данных. Поиск минимума среднеквадратичной ошибки ведется по ним. Вернее, алгоритм "фиксирует" один из параметров (пользователей), и ищет оптимальные товары.

2. Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров 


* Фактор "новизны". Например, T/t, где T - гиперпараметр, кол-во дней. t - кол-во дней с момента появления товара в магазине.
* Сезонность. Довольно глупо рекоммендовать лыжи в середине весны.
* кол-во покупок товаров "из той же категории" - для этого потребуется "подтянуть" данные о товарах, группировать по категориям и считать суммарное кол-во покупок. Но если пользователь купил уже пять квадрокоптеров, тема его явно "цепляет", и нужно для этого юзера всем коптерам установить высокий вес. Изначально же этого не видно - все куплены в единственном экземпляре, и никак не отличимы, например, от единственного фотоаппарата, купленного тем же пользователем.

3. Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)


1. Трафик. Своеобразная "цена" за сжатость данных.
2. Считает лишь факт взаимодействия пользователя с товаром. Некоторые элементы очень популярны, они получат неоправданно высокое "внимание" и кол-во выделенной памяти.
3. Минимизирует квадрат потерь - не всегда это оптимально.

4. Мы рассматривали bm_25_weight. 
Опишите, как он работает. Как сделать рекомендации только на основе bm_25? (Можно и нужно пользоваться любыми источниками, приложите на них ссылки). Какие еще способы перевзвешивания user-item матрицы Вы знаете / можете предложить (ещё 2-3 способа)?

Алгоритм оперирует параметром "встречаемости". Например, молоко покупают очень многие люди. Но это как раз снижает "ценность" продукта для рекоммендации. Мы бы хотели делать это персонально, и сводить клиента с интересующими его товарами, а не просто "впаривать" то, что покупается по-определению.

Если мы разделим q(m) (пользователи, покупающие молоко) на общее кол-во пользователей N - получим "вероятность" товара быть случайно купленным.
Чем меньше это число, тем более "уникальный" товар, и он лучше характеризует пользователя. Чтобы получить зависимость "чем больше тем больше", перевернем дробь N/q(m).

Чтобы сгладить кривую, и не "возносить до небес" редкие товары, возьмем логарифм. $\lg{\frac{N}{q(m)}}$

Итоговая вероятность того, что пользователь купил несколько разных товаров, будет равна произведению вероятностей покупки каждого из них. Поскольку вероятности у нас под логарифмом, будет сумма логарифмов.
$$\sum_{i=1}^n\lg{\frac{N}{q(m_i)}}$$

Это упрощенная реализация IDF в bm_25_weight. Именно тот механизм, который помогает перевзвесить матрицу, с учетом "уникальности запросов". Если юзер покупает зеркалку Кодак и объективы к нему, то это - не случайность, тогда как хлеб и молоко - распространенная "пара", и потому менее ценная для предсказаний.  


## Практическая часть


In [1]:
import implicit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from implicit.evaluation import precision_at_k

import warnings
warnings.filterwarnings('ignore')

In [2]:
def precision_at_k(recommended_list, bought_list, k=5):

    if isinstance(bought_list, str):
        str_list = bought_list[1:-1].split(', ')
        bought_list = [int(item) for item in str_list]
    if isinstance(recommended_list, str):
        str_list = recommended_list[1:-1].split(', ')
        recommended_list = [int(item) for item in str_list]
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    

    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)  
    

    
    precision = flags.sum() / len(recommended_list)    
    
    return precision

In [3]:
data = pd.read_csv('../../data/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [4]:
result = pd.read_csv('../../data/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[846312, 9823000, 900243, 10309250, 588311]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13511611, 1021529, 928537, 956887, 908361]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


### Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). Обучите алгоритм для различных $C$. В качестве результата приведите таблицу: матрица весов - результат на train и validation.
Сделате качественные выводы.


### Оптимизация гипперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.GridSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:
1. Значение параметра - время обучения 
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

**P.S.** Не пишите отписки в качестве выводов. Мне интресены Ваши рассуждения, трудности, с которыми Вы сталкнулись и что-то, что Вас удивило. Если выводы контринтуитивны - напишите об этом, в этом нет ничего страшного!

In [5]:
item_features = pd.read_csv('../../data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(5)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [6]:
# Категории товаров
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [7]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [8]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [9]:

# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [11]:
csr_mat = csr_matrix(user_item_matrix).T.tocsr()

In [12]:
bm25_weight_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 
tfidf_matrix = tfidf_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [13]:
#  CosineSimilarity
mu = user_item_matrix.mean().mean() # находим среднюю оценку всех товаров
ru_overline = user_item_matrix.mean(axis=1)# находим среднюю оценку данного пользователя
cosine_matrix = (user_item_matrix.T - ru_overline.values - mu).T

In [14]:
max_tfidf = tfidf_matrix.astype(float).max() 

cos_tfidf_matrix= (tfidf_matrix.astype(float) / max_tfidf) + np.array(cosine_matrix) 

In [15]:
matrixs = {'user_item':user_item_matrix, 'bm25':bm25_weight_matrix, 'tfidf':tfidf_matrix, 'cosine':cosine_matrix, 'cos&tfidf':cos_tfidf_matrix}

In [16]:
regularization = [ 0.01, 0.1, 1, 10, 100, 1000]
n_factors = [10, 15, 20, 50, 100]

In [17]:
for matrix_name in matrixs:
    for reg in regularization:
        for n_factor in n_factors:
            model = AlternatingLeastSquares(factors=n_factor, #k
                                    regularization=reg,
                                    iterations=15, 
                                    calculate_training_loss=True, 
                                    num_threads=4)

            model.fit(csr_matrix(matrixs[matrix_name]).T.tocsr(),  # На вход item-user matrix
                                    show_progress=True)

            recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                                    user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                                    N=5, # кол-во рекомендаций 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)
            result[f'ALS_{matrix_name}_{n_factor}_{reg}'] = result['user_id'].\
                apply(lambda x: [id_to_itemid[rec[0]] for rec in recs])
        

In [18]:
report = pd.DataFrame(
    [
        (model_, round(result.apply(lambda row: precision_at_k(recommended_list=row[str(model_)], bought_list=row['actual']), axis=1).mean(), 5)) for model_ in list(result)[2:]
    ], 
    columns=['model name', 'precision@k']
)

In [19]:
report.groupby('precision@k').get_group(report['precision@k'].max())

,model name,precision@k
0,ALS_user_item_10_0.01,0.17963
1,ALS_user_item_15_0.01,0.17963
5,ALS_user_item_10_0.1,0.17963
6,ALS_user_item_15_0.1,0.17963
10,ALS_user_item_10_1,0.17963
...,...,...
145,ALS_cos&tfidf_10_1000,0.17963
146,ALS_cos&tfidf_15_1000,0.17963
147,ALS_cos&tfidf_20_1000,0.17963
148,ALS_cos&tfidf_50_1000,0.17963


Видно, что 0.17963 - своеобразный "потолок", в который уперся алгоритм ALS на наших данных. При этом достигается он достаточно легко, при разных гиперпараметрах и способах взвешивания.
